### Входные данные

У вас имеется поток данных (генератор data_stream). Поля это случайные величины - так сделано для упрощения генерации данных. Есть три поля (названы по уровню сложности задания)

### Задание
##### Мотивация:
У вас есть куча временных рядов, вы хотите научиться предсказывать следующее значение по 1000 предыдущим. 1000 признаков окна это слишком много, однако вы решили заменить их 5ю: средним, дисперсией, минимумом, медианой и максимумом. Однако, все эти признаки надо подсчитать, причём хочется уметь это делать быстро (в течение часа)
##### Для каждого поля нужно сделать следующее:

1. Пробежаться по данным окном размера 1000 (окно сдвигается на 1, то есть следующее окно пересекается с предыдущим по 999 элементам).

2. Для каждого окна посчитайте среднее значение поля и его дисперсию. Делайте yield этих значений, получая генератор tuple. 

3. Для каждого окна найдине минимум, медиану и максимум в нём. Делайте yield этих значений, получая генератор tuple. 

Ответом, который нужно будет засабмитить в гугл форму, является среднее значение tuple по получившемуся потоку, округлённое до **2го знака**.

### Замечания

1. Обратите внимания как генерируются поля. Постарайтесь понять особенность каждого поля и как это можно использовать. Желательно, чтобы для каждого поля у вас было своё решение, максимально эффективно использующее знание об этом поле.
2. Полезные библиотеки: itertools, numpy, collections + всё что найдёте в интернете и можно поставить через pip install
3. **Медианой отсортированного массива arr считайте значение arr[len(arr) // 2]**



Если измерять время работы функций временем работы функции example, то примерное время работы такое:
Одновременно среднее, дисперсия - 1 минута
Одновременно минимум, максимум и медиана:easy - 3 минуты
medium - 6 минут
nightmare - 6 минут


#### Генерация данных

In [1]:
from collections import namedtuple
import random

Record = namedtuple('Record', 'easy medium nightmare')

def data_stream():
    random_generator = random.Random(42)
    easy = 0
#     for _ in range(10000000):
    for _ in range(5000):
        easy += random_generator.randint(0, 2) 
        medium = random_generator.randint(0, 256 - 1)
        nightmare = random_generator.randint(0, 1000000000 - 1)
        
        yield Record(
            easy=easy,
            medium=medium,
            nightmare=nightmare
        )
        
def easy_stream():
    for record in data_stream():
        yield record.easy
        
def medium_stream():
    for record in data_stream():
        yield record.medium
        
def nightmare_stream():
    for record in data_stream():
        yield record.nightmare

#### Подсчёт среднего значения tuple по потоку

In [2]:
import numpy as np

def get_tuple_stream_mean(stream, number_of_values):
    result = np.zeros(number_of_values, dtype='object')
    count = 0. 
    for streamed_tuple in stream:
        result += streamed_tuple
        count += 1
    return ['{:0.2f}'.format(x) for x in result / count]

In [3]:
%%time
def example(stream):
    for value in stream:
        yield (value, value + 10)
print(get_tuple_stream_mean(example(medium_stream()), 2))

['128.33', '138.33']
CPU times: user 107 ms, sys: 3.64 ms, total: 110 ms
Wall time: 107 ms


# Chunks

In [9]:
def chunks(stream):
    i = 0
    arr = []
    for value in stream:
        if i < 1000:
            i += 1
            arr.append(value)
        else:
            yield arr
            arr.pop(0)
            arr.append(value)
    yield arr

In [10]:
e = chunks(easy_stream())

In [13]:
len(next(e))

1000

# Mean, std

In [14]:
def mean_std(stream):
    for window in stream:
        yield np.mean(window), np.std(window)

In [15]:
%%time
print('easy mean, std:', get_tuple_stream_mean(mean_std(chunks(easy_stream())), 2))

easy.mean: ['2546.35', '292.90']
CPU times: user 718 ms, sys: 0 ns, total: 718 ms
Wall time: 717 ms


In [16]:
%%time
print('medium mean, std:', get_tuple_stream_mean(mean_std(chunks(medium_stream())), 2))

easy.mean: ['127.88', '74.08']
CPU times: user 718 ms, sys: 0 ns, total: 718 ms
Wall time: 719 ms


In [17]:
%%time
print('nightmare mean, std:', get_tuple_stream_mean(mean_std(chunks(nightmare_stream())), 2))

easy.mean: ['495946796.50', '287288773.73']
CPU times: user 747 ms, sys: 3.35 ms, total: 751 ms
Wall time: 750 ms


# Min, Max, Median

In [22]:
def easy_min_max_median(stream):
    for window in stream:
        yield window[0], window[-1], window[len(window) // 2]
        
def min_max_median(stream):
    for window in stream:
        window = np.sort(window)
        yield window[0], window[-1], window[len(window) // 2]

In [25]:
%%time
print('easy min, max, median:', get_tuple_stream_mean(easy_min_max_median(chunks(easy_stream())), 3))

easy min, max, median: ['2039.03', '3052.46', '2547.13']
CPU times: user 51.6 ms, sys: 3.66 ms, total: 55.2 ms
Wall time: 57.3 ms


In [26]:
%%time
print('medium min, max, median:', get_tuple_stream_mean(min_max_median(chunks(medium_stream())), 3))

medium min, max, median: ['0.00', '255.00', '129.07']
CPU times: user 444 ms, sys: 0 ns, total: 444 ms
Wall time: 450 ms


In [27]:
%%time
print('nightmare min, max, median:', get_tuple_stream_mean(min_max_median(chunks(nightmare_stream())), 3))

nightmare min, max, median: ['1227005.20', '999371320.32', '495885511.51']
CPU times: user 461 ms, sys: 368 µs, total: 462 ms
Wall time: 462 ms
